In [1]:
import hominin
from hominin import model_zoo, utils, data_processor
import yaml

import tensorflow as tf

2023-12-14 11:14:50.145140: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-14 11:14:50.982884: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-12-14 11:14:50.982958: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-12-14 11:14:50.982965: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nv

In [2]:
import importlib

importlib.reload(model_zoo)

<module 'hominin.model_zoo' from '/home/chandana/projects/hominin/hominin/model_zoo.py'>

In [2]:
config_path = '/home/chandana/projects/hominin/experiments/lenti-mpra/cam/standard/softconv/attention_pool/pool_size=0/elu/trial_0/config.yaml'
config = yaml.full_load(open(config_path))

dp = data_processor.DataProcessor(dataset='hepg2', subsample=False)

x_train, y_train = dp.load_data("train")
x_valid, y_valid = dp.load_data("valid")
x_test, y_test = dp.load_data("test")
# sample = x_train[:1]

(97915, 1)
(13987, 1)
(27975, 1)


In [3]:
from tensorflow.keras import Model
from tensorflow.keras.layers import Input, Conv1D, MaxPooling1D, Flatten, Dense, Dropout, BatchNormalization, ReLU, LayerNormalization


def standard(input_shape, output_shape):
    inputs = Input(shape=(input_shape[1], input_shape[2]), name="input")
    layer = Conv1D(250, kernel_size=7, strides=1, activation='relu', name="conv1")(inputs)  # 250 7 relu
    layer = BatchNormalization()(layer)
    layer = Conv1D(250, 8, strides=1, activation='softmax', name="conv2")(layer)  # 250 8 softmax
    layer = BatchNormalization()(layer)
    layer = MaxPooling1D(pool_size=2, strides=None, name="maxpool1")(layer)
    layer = Dropout(0.1)(layer)
    layer = Conv1D(250, 3, strides=1, activation='softmax', name="conv3")(layer)  # 250 3 softmax
    layer = BatchNormalization()(layer)
    layer = Conv1D(100, 2, strides=1, activation='softmax', name="conv4")(layer)  # 100 3 softmax
    layer = BatchNormalization()(layer)
    layer = MaxPooling1D(pool_size=1, strides=None, name="maxpool2")(layer)
    layer = Dropout(0.1)(layer)
    layer = Flatten()(layer)
    layer = Dense(300, activation='sigmoid')(layer)  # 300
    layer = Dropout(0.3)(layer)
    layer = Dense(200, activation='sigmoid')(layer)  # 300
    predictions = Dense(output_shape[1], activation='linear')(layer)
    model = Model(inputs=inputs, outputs=predictions)
    return (model)

In [7]:
model = standard(input_shape=(_, 230, 4), output_shape=(_, 1))

optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
loss = 'mse'
model.compile(optimizer=optimizer, loss=loss)


history = model.fit(
            x_train, y_train,
            epochs=100,
            batch_size=64, # need to explore which batch size is possible! (64 for citra)
            shuffle=True,
            validation_data=(x_valid, y_valid),
            callbacks=[
                 tf.keras.callbacks.EarlyStopping(
                    monitor="val_mse",
                    patience=10,
                    verbose=1,
                    mode="max",
                    restore_best_weights=True
                ),
            ]
)

Epoch 1/100
1530/1530 [==============================] - 32s 16ms/step - loss: 0.4604 - val_loss: 0.3888
Epoch 2/100
1530/1530 [==============================] - 24s 16ms/step - loss: 0.3797 - val_loss: 0.3596
Epoch 3/100
1530/1530 [==============================] - 24s 16ms/step - loss: 0.3454 - val_loss: 0.3486
Epoch 4/100
1530/1530 [==============================] - 25s 16ms/step - loss: 0.3149 - val_loss: 0.3921
Epoch 5/100
1530/1530 [==============================] - 24s 16ms/step - loss: 0.2889 - val_loss: 0.3557
Epoch 6/100
1530/1530 [==============================] - 25s 16ms/step - loss: 0.2687 - val_loss: 0.3362
Epoch 7/100
1530/1530 [==============================] - 25s 16ms/step - loss: 0.2438 - val_loss: 0.3667
Epoch 8/100
1530/1530 [==============================] - 25s 16ms/step - loss: 0.2255 - val_loss: 0.3689
Epoch 9/100
1530/1530 [==============================] - 24s 16ms/step - loss: 0.2062 - val_loss: 0.3377
Epoch 10/100
1530/1530 [==============================]

1530/1530 [==============================] - 25s 16ms/step - loss: 0.0759 - val_loss: 0.3437
Epoch 29/100
1530/1530 [==============================] - 25s 16ms/step - loss: 0.0731 - val_loss: 0.3513
Epoch 30/100
1530/1530 [==============================] - 25s 16ms/step - loss: 0.0705 - val_loss: 0.3422
Epoch 31/100
1530/1530 [==============================] - 24s 16ms/step - loss: 0.0692 - val_loss: 0.3467
Epoch 32/100
1530/1530 [==============================] - 25s 16ms/step - loss: 0.0663 - val_loss: 0.3676
Epoch 33/100
1530/1530 [==============================] - 25s 16ms/step - loss: 0.0646 - val_loss: 0.3374
Epoch 34/100
1530/1530 [==============================] - 24s 16ms/step - loss: 0.0632 - val_loss: 0.3933
Epoch 35/100
1530/1530 [==============================] - 25s 16ms/step - loss: 0.0618 - val_loss: 0.3458
Epoch 36/100
1530/1530 [==============================] - 25s 16ms/step - loss: 0.0610 - val_loss: 0.3864
Epoch 37/100
1530/1530 [==============================] - 2

Epoch 56/100
1530/1530 [==============================] - 25s 16ms/step - loss: 0.0409 - val_loss: 0.3480
Epoch 57/100
1530/1530 [==============================] - 25s 16ms/step - loss: 0.0405 - val_loss: 0.3968
Epoch 58/100
1530/1530 [==============================] - 25s 16ms/step - loss: 0.0399 - val_loss: 0.3752
Epoch 59/100
1530/1530 [==============================] - 25s 16ms/step - loss: 0.0386 - val_loss: 0.3342
Epoch 60/100
1530/1530 [==============================] - 25s 16ms/step - loss: 0.0388 - val_loss: 0.3912
Epoch 61/100
1530/1530 [==============================] - 25s 16ms/step - loss: 0.0375 - val_loss: 0.3529
Epoch 62/100
1530/1530 [==============================] - 25s 16ms/step - loss: 0.0375 - val_loss: 0.3503
Epoch 63/100
1530/1530 [==============================] - 25s 16ms/step - loss: 0.0375 - val_loss: 0.3562
Epoch 64/100
1530/1530 [==============================] - 25s 16ms/step - loss: 0.0364 - val_loss: 0.4292
Epoch 65/100
1530/1530 [======================

1530/1530 [==============================] - 24s 16ms/step - loss: 0.0297 - val_loss: 0.4070
Epoch 84/100
1530/1530 [==============================] - 25s 16ms/step - loss: 0.0293 - val_loss: 0.3479
Epoch 85/100
1530/1530 [==============================] - 25s 16ms/step - loss: 0.0288 - val_loss: 0.4025
Epoch 86/100
1530/1530 [==============================] - 25s 16ms/step - loss: 0.0285 - val_loss: 0.3276
Epoch 87/100
1530/1530 [==============================] - 25s 16ms/step - loss: 0.0285 - val_loss: 0.4034
Epoch 88/100
1530/1530 [==============================] - 25s 16ms/step - loss: 0.0280 - val_loss: 0.3417
Epoch 89/100
1530/1530 [==============================] - 25s 16ms/step - loss: 0.0288 - val_loss: 0.3550
Epoch 90/100
1530/1530 [==============================] - 25s 16ms/step - loss: 0.0276 - val_loss: 0.3283
Epoch 91/100
1530/1530 [==============================] - 25s 16ms/step - loss: 0.0278 - val_loss: 0.3554
Epoch 92/100
1530/1530 [==============================] - 2

In [8]:
mse, pcc, scc = utils.evaluate_model(model, x_test, y_test, "Dev")
print(pcc)

438/438 [==============================] - 2s 4ms/step
Dev MSE = 0.347
Dev PCC = 0.693
Dev SCC = 0.683
0.692998602794394


In [3]:
config_1 = {
    'base_model': 'cam',
    'conv1_activation': 'exponential',
    'conv1_batchnorm': True,
    'conv1_dropout': 0.1,
    'conv1_filters': 128,
    'conv1_kernel_size': 19,
    'conv1_pool_size': 2,
    'conv_layer_type': 'standard',
    'dense_activation': 'relu',
    'dense_batchnorm': True,
    'dense_dropout': [0.3, 0.3],
    'dense_units': [256, 256],
    'input_shape': (249, 4),
    'mha_d_model': 96,
    'mha_dropout': 0.2,
    'mha_head_type': 'pooled',
    'mha_heads': 8,
    'mha_layernorm': True,
    'motif_pooling_type': 'softconv',
    'output_activation': 'linear',
    'output_shape': 2,
    'spatial_pooling_type': 'attention_pool'
}

In [4]:
model_1 = model_zoo.build_model(**config_1)

model_1

standard 128 19
standard 128 19
Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


In [7]:
config_2 = {
    'base_model': 'nam',
    'conv1_activation': 'exponential',
    'conv1_batchnorm': True,
    'conv1_dropout': 0.1,
    'conv1_filters': 128,
    'conv1_kernel_size': 19,
    'conv1_pool_size': 2,
    'conv_layer_type': 'pairwise',
    'dense_activation': 'relu',
    'dense_batchnorm': True,
    'dense_dropout': [0.3, 0.3],
    'dense_units': [256, 256],
    'input_shape': (249, 4),
    'mha_d_model': 96,
    'mha_dropout': 0.2,
    'mha_head_type': 'pooled',
    'mha_heads': 8,
    'mha_layernorm': True,
    'motif_pooling_type': 'softconv',
    'output_activation': 'linear',
    'output_shape': 2,
    'spatial_pooling_type': 'attention_pool'
}

In [8]:
model_2 = model_zoo.build_model(**config_2)

model_2

pairwise 128 19
pairwise 128 19
Shape of nn: (None, 124, 128, 256)


In [9]:
model = model_2

optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
loss = 'mse'
# metrics = [utils.Spearman, utils.pearson_r]
model.compile(optimizer=optimizer, loss=loss)


history = model.fit(
            x_train, y_train,
            epochs=1,
            batch_size=64, # need to explore which batch size is possible! (64 for citra)
            shuffle=True,
)


 199/1250 [===>..........................] - ETA: 2:01 - loss: 3.3270

KeyboardInterrupt: 

In [17]:
python execute_hominin.py --config_file=/home/chandana/temp/config.yaml --dataset_name=deepstarr --smoke_test=True  --log_wandb=False --gpu=1

64

In [41]:
output_1 = model_1(sample)

In [42]:
output_2 = model_2(sample)

In [43]:
output_1.shape, output_2.shape

(TensorShape([1, 2]), TensorShape([1, 2]))

In [ ]:
layer